# Library

In [23]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Python path
import sys
base_folder = 'DataScience'
location_base = os.path.join(os.getcwd().split(base_folder)[0], base_folder)
location_module = [os.path.join(location_base, 'Module')] 
for each in location_module:
    if each not in sys.path:
        sys.path.append(each)

# Auto reload of library
%reload_ext autoreload
%autoreload 2

from import_KK import *
DeviceStrategy_CPU()
from preprocessing_KK import *
from preprocessing_text_KK import * ##
from visualization_KK import * ##
from algorithm_KK import *
from evaluation_KK import *

# Hyperparameters

In [24]:
# 하이퍼파라미터
FOLDER_LOCATION = os.path.join(os.getcwd(), 'Data', '빅카인즈')
FOLDER_NAME = True
CATEGORY = ['경제', '사회', '문화', '국제']
IMAGE_LOCATION = os.path.join('.', 'Data', 'baby-icon_ver1.png')
COLNAME_CATEGORY = '일자'
COLNAME_MINING = '제목'
CONSERVATIVE = ['동아일보', '중앙일보', '조선일보', '매일경제', '한국경제']
PROGRESSIVE = ['한겨례', '경향신문', '머니투데이']
################
test_criteria = '2023-01-01'
SEQUENCE = 5
Y_SCALING = True
MOVING_TYPE = 'sliding'    # 'sliding', 'expanding'
TRAIN_WINDOW = 100
FORECASTING_PERIOD = 4+24
################
# metrics.SCORERS.keys(): 'neg_mean_squared_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error'
LOSS_ML = 'neg_mean_squared_error' 
PARAMS_BAG = None
# PARAMS_BAG = {'n_estimators': [30, 50, 90],     
#           'max_depth': [20, 30, 40, 50],  
#           'max_leaf_nodes': [5, 10, 15]}
PARAMS_BOOST = None
# PARAMS_BOOST = {'n_estimators': [30, 50, 90],    
#           'max_depth': [20, 30, 40, 50],   
#           'num_leaves': [5, 10, 15],
#           'min_child_weight': [3, 5, 7],
#           'learning_rate': [0.1, 0.01],
#           'force_col_wise': [False], 'force_row_wise': [True]}
CV_SPLITS = 5
################
KERNEL_SIZE = 2
STRIDE = 1
POOL_SIZE = 1
POOL_STRIDE = 1
HIDDEN_ACTIVATION = 'relu'
OUTPUT_ACTIVATION = 'linear'
REGULARIZER = None
DROPOUT_RATIO = 0.2
MODEL_SUMMARY = False
LOSS = 'mse'
LEARNING_RATE = 0.01
OPTIMIZER = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
METRICS = ['mse']
VALIDATION_SPLIT = 0.2
VALIDATION_DATA = None
BATCH_SIZE = 32
EPOCHS = 500
VERBOSE = 0
################
EARLYSTOP_PATIENT = EPOCHS*0.4
MONITOR = 'val_loss'
LEARNING_PLOT = False
PLOT_TITLE = "How People's Interest Changes Over Time"
PLOT_XLABEL = 'Time'
PLOT_YLABEL = 'Interest Forecasting'
################


# Data Preprocessing

## Google Trend

In [25]:
# 데이터로딩
file_location = os.path.join(os.getcwd(), 'Data', 'ageism_survey_KK.xlsx')
df_gt = pd.read_excel(file_location, sheet_name='GT_Trend', index_col='Time')
df_gt_global = df_gt[['Ageism']].copy()
df_gt_local = df_gt[['고령화']].copy()

## BigKinds

In [26]:
# ### 하위의 모든 데이터 결합하여 출력
# def get_data_from_path(folder_location, folder_name=False, concat_axis='row'):
#     # path_folder 하위의 모든 폴더위치와 내부 file 출력
#     df = pd.DataFrame()
#     print('Getting data from', len(os.listdir(folder_location)), 'folders...')
#     for (path, dir, files) in os.walk(folder_location):
#         for file in tqdm(files):
#             path_file = os.path.join(path, file)

#             ## 데이터 로딩
#             if path_file.split('.')[1] == 'xlsx':
#                 df_sub = pd.read_excel(path_file)
#             elif path_file.split('.')[1] == 'csv':
#                 df_sub = pd.read_csv(path_file)
                
#             ## 키워드 태깅 여부
#             if folder_name:
#                 df_sub['Folder_Name'] = os.path.basename(path)
            
#             ## 정리
#             if concat_axis == 'col':
#                 df = pd.concat([df, df_sub], axis=1)
#             elif concat_axis == 'row':
#                 df = pd.concat([df, df_sub], axis=0)
                
#     return df

# # 데이터로딩
# df_news = get_data_from_path(FOLDER_LOCATION, folder_name=FOLDER_NAME)
# ## 중복 처리
# df_news.drop_duplicates(subset=['뉴스 식별자', '언론사', '제목'], inplace=True, ignore_index=True)
# ## 날짜 인식
# df_news['일자'] = pd.to_datetime(df_news['일자'].astype(str))
# ## 월별 트랜드용 데이터분리
# df_news_rising = df_news[['Folder_Name', '일자', '제목']].copy()
# df_news_rising['일자'] = pd.to_datetime(df_news_rising['일자'].dt.strftime('%Y-%m'))
# df_news_rising = df_news_rising.groupby(['Folder_Name', '일자'])['제목'].count().unstack(level=0).fillna(0)
# ## 연도만 남기기
# df_news['일자'] = df_news['일자'].dt.year
# ## 카테고리 필터
# category_filter = [each for each in df_news['통합 분류1'].unique() if each.split('>')[0] in CATEGORY]
# print(category_filter)
# df_news = df_news[df_news['통합 분류1'].apply(lambda x: x in category_filter)].reset_index().iloc[:,1:]
# ## 전처리
# df_news['제목'] = df_news['제목'].apply(lambda x: text_preprocessor(x, del_number=False, del_bracket_content=False))
# ## 결측치 및 빈문자 제거
# df_news = df_news[~df_news['제목'].isnull()].reset_index().iloc[:,1:].copy()
# df_news = df_news[df_news['제목'].str.len() != 0].reset_index().iloc[:,1:]
# ## 언론사 분리
# df_newsc = df_news[df_news['언론사'].isin(CONSERVATIVE)].reset_index()
# df_newsp = df_news[df_news['언론사'].isin(PROGRESSIVE)].reset_index()
# ## 날짜 변환 데이터
# df_news_era, df_newsc_era, df_newsp_era = df_news.copy(), df_newsc.copy(), df_newsp.copy()
# df_news_era['일자'] = df_news_era['일자'].apply(lambda x: '2013 ~ 2017' if x in [2013, 2014, 2015, 2016, 2017] 
#                                                                          else '2018 ~ 2023')
# df_newsc_era['일자'] = df_newsc_era['일자'].apply(lambda x: '2013 ~ 2017' if x in [2013, 2014, 2015, 2016, 2017] 
#                                                                          else '2018 ~ 2023')
# df_newsp_era['일자'] = df_newsp_era['일자'].apply(lambda x: '2013 ~ 2017' if x in [2013, 2014, 2015, 2016, 2017] 
#                                                                          else '2018 ~ 2023')
# ## 저장
# df_news_rising.to_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_news_rising.csv'), index=False, encoding='utf-8-sig')
# df_news.to_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_news.csv'), index=False, encoding='utf-8-sig')
# df_newsc.to_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_newsc.csv'), index=False, encoding='utf-8-sig')
# df_newsp.to_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_newsp.csv'), index=False, encoding='utf-8-sig')
# df_news_era.to_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_news_era.csv'), index=False, encoding='utf-8-sig')
# df_newsc_era.to_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_newsc_era.csv'), index=False, encoding='utf-8-sig')
# df_newsp_era.to_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_newsp_era.csv'), index=False, encoding='utf-8-sig')

In [27]:
## 불러오기
df_news_rising = pd.read_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_news_rising.csv'))
df_news = pd.read_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_news.csv'))
df_newsc = pd.read_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_newsc.csv'))
df_newsp = pd.read_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_newsp.csv'))
df_news_era = pd.read_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_news_era.csv'))
df_newsc_era = pd.read_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_newsc_era.csv'))
df_newsp_era = pd.read_csv(os.path.join(r'C:\Users\KK\Desktop', 'df_newsp_era.csv'))

## Word Frequency

- **데이터:** 구글 뉴스 + 네이버 뉴스
- **카테고리:** 경세 + 사회 + 문화 + 국제

In [28]:
print('전체 언론사:', df_news['언론사'].unique())
print('\n보수 언론사:', df_newsc['언론사'].unique())
print('\n진보 언론하:', df_newsp['언론사'].unique())

전체 언론사: ['아주경제' '동아일보' '세계일보' '머니투데이' '중앙일보' '서울경제' '헤럴드경제' '문화일보' '국민일보' '서울신문'
 '디지털타임스' '한국경제' '아시아경제' '매일경제' '내일신문' 'SBS' '파이낸셜뉴스' '경향신문' '전자신문' '한겨레'
 'YTN' '한국일보' 'MBC' 'OBS' 'KBS' '조선일보']

보수 언론사: ['동아일보' '중앙일보' '한국경제' '매일경제' '조선일보']

진보 언론하: ['머니투데이' '경향신문']


In [29]:
# # 연도데이터 기준 전처리
# wf_soy, waf_soy, wf_tf, waf_tf = preprocessing_wordfreq(df_news, colname_target='제목', colname_category='일자', 
#                          save_local=True)
# wfc_soy, wafc_soy, wfc_tf, wafc_tf = preprocessing_wordfreq(df_newsc, colname_target='제목', colname_category='일자', 
#                          save_local=True,
#                          save_name_list=['word_freq_soynlp_c.csv', 'wordadj_freq_soynlp_c.csv', 
#                                          'word_freq_tfidf_c.csv', 'wordadj_freq_tfidf_c.csv'])
# wfp_soy, wafp_soy, wfp_tf, wafp_tf = preprocessing_wordfreq(df_newsp, colname_target='제목', colname_category='일자', 
#                          save_local=True,
#                          save_name_list=['word_freq_soynlp_p.csv', 'wordadj_freq_soynlp_p.csv', 
#                                          'word_freq_tfidf_p.csv', 'wordadj_freq_tfidf_p.csv'])

# # 연도그룹데이터 기준 전처리
# wf_era_soy, waf_era_soy, wf_era_tf, waf_era_tf = preprocessing_wordfreq(df_news_era, colname_target='제목', colname_category='일자', 
#                          save_local=True,
#                          save_name_list=['word_freq_soynlp_era.csv', 'wordadj_freq_soynlp_era.csv', 
#                                          'word_freq_tfidf_era.csv', 'wordadj_freq_tfidf_era.csv'])
# ## 안되면 아래줄 실행
# wf_era_tf = wf_tf.copy()
# wf_era_tf.category.apply(lambda x: '2013 ~ 2017' if x in ['2013', '2014', '2015', '2016', '2017']
#                                                   else '2018 ~ 2023')
# wf_era_tf = wf_era_tf.groupby(list(wf_era_tf.columns[:2])).mean().reset_index()
# waf_era_tf = pd.DataFrame()
# for category in tqdm(sorted(df_news_era[COLNAME_CATEGORY].unique())):
#     df_sub = df_news_era[df_news_era[COLNAME_CATEGORY] == category]
#     waf_era = preprocessing_adjwordcount(wf_era_tf[['word']], df_sub[COLNAME_MINING], num_showkeyword=5)
#     waf_era['category'] = str(category)
#     waf_era = waf_era[['category']+list(waf_era.columns[:-1])]
#     waf_era_tf = pd.concat([waf_era_tf, waf_era], axis=0, ignore_index=True)
# save_name = os.path.join(os.getcwd(), 'Data', 'word_freq_tfidf_era.csv')
# wf_era_tf.to_csv(save_name, index=False, encoding='utf-8-sig')
# save_name = os.path.join(os.getcwd(), 'Data', 'wordadj_freq_tfidf_era.csv')
# waf_era_tf.to_csv(save_name, index=False, encoding='utf-8-sig')
# ######################
# wfc_era_soy, wafc_era_soy, wfc_era_tf, wafc_era_tf = preprocessing_wordfreq(df_newsc_era, colname_target='제목', colname_category='일자', 
#                          save_local=True,
#                          save_name_list=['word_freq_soynlp_erac.csv', 'wordadj_freq_soynlp_erac.csv', 
#                                          'word_freq_tfidf_erac.csv', 'wordadj_freq_tfidf_erac.csv'])
# wfp_era_soy, wafp_era_soy, wfp_era_tf, wafp_era_tf = preprocessing_wordfreq(df_newsp_era, colname_target='제목', colname_category='일자', 
#                          save_local=True,
#                          save_name_list=['word_freq_soynlp_erap.csv', 'wordadj_freq_soynlp_erap.csv', 
#                                          'word_freq_tfidf_erap.csv', 'wordadj_freq_tfidf_erap.csv'])

In [30]:
# 불러오기
save_name_list=['word_freq_soynlp.csv', 'wordadj_freq_soynlp.csv', 
                'word_freq_tfidf.csv', 'wordadj_freq_tfidf.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wf_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
waf_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wf_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
waf_tfidf = pd.read_csv(save_name)

save_name_list=['word_freq_soynlp_c.csv', 'wordadj_freq_soynlp_c.csv', 
                'word_freq_tfidf_c.csv', 'wordadj_freq_tfidf_c.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wfc_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
wafc_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wfc_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
wafc_tfidf = pd.read_csv(save_name)

save_name_list=['word_freq_soynlp_p.csv', 'wordadj_freq_soynlp_p.csv', 
                'word_freq_tfidf_p.csv', 'wordadj_freq_tfidf_p.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wfp_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
wafp_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wfp_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
wafp_tfidf = pd.read_csv(save_name)

# 불러오기
save_name_list=['word_freq_soynlp_era.csv', 'wordadj_freq_soynlp_era.csv', 
                'word_freq_tfidf_era.csv', 'wordadj_freq_tfidf_era.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wf_era_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
waf_era_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wf_era_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
waf_era_tfidf = pd.read_csv(save_name)

save_name_list=['word_freq_soynlp_erac.csv', 'wordadj_freq_soynlp_erac.csv', 
                'word_freq_tfidf_erac.csv', 'wordadj_freq_tfidf_erac.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wfc_era_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
wafc_era_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wfc_era_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
wafc_era_tfidf = pd.read_csv(save_name)

save_name_list=['word_freq_soynlp_erap.csv', 'wordadj_freq_soynlp_erap.csv', 
                'word_freq_tfidf_erap.csv', 'wordadj_freq_tfidf_erap.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wfp_era_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
wafp_era_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wfp_era_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
wafp_era_tfidf = pd.read_csv(save_name)

## Word Correlation

In [31]:
# # 관련성 전처리
# wf_corr_soynlp = preprocessing_wordfreq_to_corr(wf_era_soynlp, df_news_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='word_corrpair_soynlp_era.csv')
# waf_corr_soynlp = preprocessing_wordfreq_to_corr(waf_era_soynlp, df_news_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='wordadj_corrpair_soynlp_era.csv')
# wf_corr_tfidf = preprocessing_wordfreq_to_corr(wf_era_tfidf, df_news_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='word_corrpair_tfidf_era.csv')
# waf_corr_tfidf = preprocessing_wordfreq_to_corr(waf_era_tfidf, df_news_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='wordadj_corrpair_tfidf_era.csv')

# wfc_corr_soynlp = preprocessing_wordfreq_to_corr(wfc_era_soynlp, df_newsc_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='word_corrpair_soynlp_era_c.csv')
# wafc_corr_soynlp = preprocessing_wordfreq_to_corr(wafc_era_soynlp, df_newsc_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='wordadj_corrpair_soynlp_era_c.csv')
# wfc_corr_tfidf = preprocessing_wordfreq_to_corr(wfc_era_tfidf, df_newsc_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='word_corrpair_tfidf_era_c.csv')
# wafc_corr_tfidf = preprocessing_wordfreq_to_corr(wafc_era_tfidf, df_newsc_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='wordadj_corrpair_tfidf_era_c.csv')

# wfp_corr_soynlp = preprocessing_wordfreq_to_corr(wfp_era_soynlp, df_newsp_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='word_corrpair_soynlp_era_p.csv')
# wafp_corr_soynlp = preprocessing_wordfreq_to_corr(wafp_era_soynlp, df_newsp_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='wordadj_corrpair_soynlp_era_p.csv')
# wfp_corr_tfidf = preprocessing_wordfreq_to_corr(wfp_era_tfidf, df_newsp_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='word_corrpair_tfidf_era_p.csv')
# wafp_corr_tfidf = preprocessing_wordfreq_to_corr(wafp_era_tfidf, df_newsp_era, colname_target=COLNAME_MINING, colname_category=COLNAME_CATEGORY, num_showkeyword=100, save_name='wordadj_corrpair_tfidf_era_p.csv')

In [32]:
# 불러오기
save_name_list=['word_corrpair_soynlp_era.csv', 'wordadj_corrpair_soynlp_era.csv', 
                'word_corrpair_tfidf_era.csv', 'wordadj_corrpair_tfidf_era.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wf_corr_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
waf_corr_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wf_corr_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
waf_corr_tfidf = pd.read_csv(save_name)

save_name_list=['word_corrpair_soynlp_era_c.csv', 'wordadj_corrpair_soynlp_era_c.csv', 
                'word_corrpair_tfidf_era_c.csv', 'wordadj_corrpair_tfidf_era_c.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wfc_corr_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
wafc_corr_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wfc_corr_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
wafc_corr_tfidf = pd.read_csv(save_name)

save_name_list=['word_corrpair_soynlp_era_p.csv', 'wordadj_corrpair_soynlp_era_p.csv', 
                'word_corrpair_tfidf_era_p.csv', 'wordadj_corrpair_tfidf_era_p.csv']
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[0])
wfp_corr_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[1])
wafp_corr_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[2])
wfp_corr_tfidf = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', save_name_list[3])
wafp_corr_tfidf = pd.read_csv(save_name)

# 연구문제0

## Ageism Global Trend and Forecasting

In [ ]:
for each in [df_gt_global, df_gt_local]:
    model, Score_te = modeling_LGBMRegressor_slidingwindow1D(each, SEQUENCE=SEQUENCE, Y_SCALING=Y_SCALING, MOVING_TYPE=MOVING_TYPE,
                                                   TRAIN_WINDOW=TRAIN_WINDOW, FORECASTING_PERIOD=FORECASTING_PERIOD,
                                                   LOSS=LOSS_ML, GRIDSEARCH_PARAMS=PARAMS_BOOST, CV_SPLITS=CV_SPLITS,
                                                   PLOT_TITLE=PLOT_TITLE, PLOT_XLABEL=PLOT_XLABEL, PLOT_YLABEL=PLOT_YLABEL)
    display(Score_te)

## Korea Trend by Keywords

In [ ]:
## 증가변수 필터
RISING_CATEG = ['고령화', '노인+경제적+자립', '노인+봉사', '노인+의료', 
                '노인+주택+문제', '노인+혐오', '돌봄서비스', '세대+갈등', '소회', 
                '안락사', '알츠하이머', '연령+통합']
df_news_rising = df_news_rising[RISING_CATEG].copy()
## 시각화
plot_timeseries_dforigin(df_news_rising, save_local=True, save_name_initial='gt_total_origin.png')
plot_timeseries_dfmeanstd(df_news_rising, scaled=True, save_local=True, save_name_initial='gt_total_scaled.png')
plot_timeseries(df_news_rising, save_local=True, save_name_initial='gt_each_origin.png')

# 연구문제1

## 과거 10년 동안의 온라인 검색어 트랜드

In [ ]:
# word freq 비교 트랜드 시각화
plot_treemap_wordfreq(wf_soynlp, num_showkeyword=100, title='Ageism by Year', plot_studio=True, save_local=True, save_name='trend_year_compare_treemap.html')
plot_sunburst_wordfreq(waf_soynlp, title='Ageism by Year', plot_studio=True, save_local=True, save_name='trend_year_compare_sunburst.html')
plot_bar_wordfreq(wf_soynlp, figsize=(30,8), num_showkeyword=20, title='Ageism by Year', save_local=True, save_name='trend_year_compare_bar_byword.png')
plot_bar_wordfreq(waf_soynlp, figsize=(30,8), num_showkeyword=20, title='Ageism by Year', save_local=True, save_name='trend_year_compare_bar_bywordadj.png')

## 고령사회 전후의 온라인 검색어 트랜드

In [ ]:
# 연도별 word freq 시각화
centrality_categ = pd.DataFrame()
for category in wf_era_soynlp[wf_era_soynlp.columns[0]].unique():
    print(category)
    wf_sub = wf_era_soynlp.groupby(wf_era_soynlp.columns[0]).get_group(category)
    wc_sub = wf_corr_soynlp[wf_corr_soynlp.category.isin([category])]
    
    plot_wordcloud(wf_sub.iloc[:,1:], mask_colorgen=False, max_words=100, mask_location=IMAGE_LOCATION, save_local=True, save_name='trend_era_wordcloud_'+'~'.join(category.split(' ~ '))+'.png')
    plot_bar_wordfreq(wf_sub.iloc[:,1:], figsize=(16,8), num_showkeyword=100, title='Ageism by Era', 
                      save_local=True, save_name='trend_era_bar_'+'~'.join(category.split(' ~ '))+'.png')
    plot_donut_wordfreq(wf_sub.iloc[:,1:], num_showkeyword=30, save_local=True, save_name='trend_era_donut_'+'~'.join(category.split(' ~ '))+'.html')
    _, centrality = plot_networkx(wf_sub, wc_sub.iloc[:,1:], 
                                  filter_criteria=0.02, plot=True, node_size='pagerank', save_local=True, save_name='trend_era_networkx_'+'~'.join(category.split(' ~ '))+'.png')
    ## 카테고리 추가
    centrality['category'] = str(category)
    centrality_categ = pd.concat([centrality_categ, centrality], axis=0, ignore_index=True)

In [ ]:
# word freq 비교 트랜드 시각화
plot_treemap_wordfreq(wf_era_soynlp, num_showkeyword=100, title='Ageism_Era', plot_studio=True, save_name='trend_era_compare_treemap_'+'~'.join(category.split(' ~ '))+'.html')
plot_sunburst_wordfreq(waf_era_soynlp, title='Ageism_Era', plot_studio=True, save_name='trend_era_compare_sunburst_'+'~'.join(category.split(' ~ '))+'.html')
plot_bar_wordfreq(wf_era_soynlp, figsize=(20,8), num_showkeyword=20, title='Ageism_Era', save_local=True, save_name='trend_era_compare_bar_byword_'+'~'.join(category.split(' ~ '))+'.png')
plot_bar_wordfreq(waf_era_soynlp, figsize=(20,8), num_showkeyword=20, title='Ageism_Era', save_local=True, save_name='trend_era_compare_bar_bywordadj_'+'~'.join(category.split(' ~ '))+'.png')

## 과거 10년 동안의 보수 및 진보 트랜드

In [ ]:
# word freq 비교 트랜드 시각화
plot_treemap_wordfreq(wfc_soynlp, num_showkeyword=100, title='Ageism by Year+Conservative', plot_studio=True, save_local=True, save_name='trend_yearc_compare_treemap.html')
plot_treemap_wordfreq(wfp_soynlp, num_showkeyword=100, title='Ageism by Year+Progressive', plot_studio=True, save_local=True, save_name='trend_yearp_compare_treemap.html')
plot_sunburst_wordfreq(wafc_soynlp, title='Ageism by Year+Conservative', plot_studio=True, save_local=True, save_name='trend_yearc_compare_sunburst.html')
plot_sunburst_wordfreq(wafp_soynlp, title='Ageism by Year+Progressive', plot_studio=True, save_local=True, save_name='trend_yearp_compare_sunburst.html')
plot_bar_wordfreq(wfc_soynlp, figsize=(30,8), num_showkeyword=20, title='Ageism by Year+Conservative', save_local=True, save_name='trend_yearc_compare_bar_byword.png')
plot_bar_wordfreq(wfp_soynlp, figsize=(30,8), num_showkeyword=20, title='Ageism by Year+Progressive', save_local=True, save_name='trend_yearp_compare_bar_byword.png')
plot_bar_wordfreq(wafc_soynlp, figsize=(30,8), num_showkeyword=20, title='Ageism by Year+Conservative', save_local=True, save_name='trend_yearc_compare_bar_bywordadj.png')
plot_bar_wordfreq(wafp_soynlp, figsize=(30,8), num_showkeyword=20, title='Ageism by Year+Progressive', save_local=True, save_name='trend_yearp_compare_bar_bywordadj.png')

## 고령사회 전후의 보수 및 진보 트랜드

In [ ]:
# 연도별 word freq 시각화
centrality_categ = pd.DataFrame()
for category in wfc_era_soynlp[wfc_era_soynlp.columns[0]].unique():
    print(category)
    wf_subc = wfc_era_soynlp.groupby(wfc_era_soynlp.columns[0]).get_group(category)
    wc_subc = wfc_corr_soynlp[wfc_corr_soynlp.category.isin([category])]
    wf_subp = wfp_era_soynlp.groupby(wfp_era_soynlp.columns[0]).get_group(category)
    wc_subp = wfp_corr_soynlp[wfp_corr_soynlp.category.isin([category])]
    
    plot_wordcloud(wf_subc.iloc[:,1:], mask_colorgen=False, max_words=100, mask_location=IMAGE_LOCATION, save_local=True, save_name='trend_erac_wordcloud_'+'~'.join(category.split(' ~ '))+'.png')
    plot_wordcloud(wf_subp.iloc[:,1:], mask_colorgen=False, max_words=100, mask_location=IMAGE_LOCATION, save_local=True, save_name='trend_erap_wordcloud_'+'~'.join(category.split(' ~ '))+'.png')
    plot_bar_wordfreq(wf_subc.iloc[:,1:], num_showkeyword=100, save_local=True, save_name='trend_erac_bar_'+'~'.join(category.split(' ~ '))+'.png')
    plot_bar_wordfreq(wf_subp.iloc[:,1:], num_showkeyword=100, save_local=True, save_name='trend_erap_bar_'+'~'.join(category.split(' ~ '))+'.png')
    plot_donut_wordfreq(wf_subc.iloc[:,1:], num_showkeyword=30, save_local=True, save_name='trend_erac_donut_'+'~'.join(category.split(' ~ '))+'.html')
    plot_donut_wordfreq(wf_subp.iloc[:,1:], num_showkeyword=30, save_local=True, save_name='trend_erap_donut_'+'~'.join(category.split(' ~ '))+'.html')
    _, centrality = plot_networkx(wf_subc, wc_subc.iloc[:,1:], 
                                  filter_criteria=0.025, plot=True, node_size='pagerank', save_local=True, save_name='trend_erac_networkx_'+'~'.join(category.split(' ~ '))+'.png')
    _, centrality = plot_networkx(wf_subp, wc_subp.iloc[:,1:], 
                                  filter_criteria=0.025, plot=True, node_size='pagerank', save_local=True, save_name='trend_erap_networkx_'+'~'.join(category.split(' ~ '))+'.png')


In [ ]:
# word freq 비교 트랜드 시각화
plot_treemap_wordfreq(wfc_era_soynlp, num_showkeyword=100, title='Ageism_Era_Conservative', plot_studio=True, save_local=True, save_name='trend_erac_compare_treemap.html')
plot_treemap_wordfreq(wfp_era_soynlp, num_showkeyword=100, title='Ageism_Era_Progressive', plot_studio=True, save_local=True, save_name='trend_erap_compare_treemap.html')
plot_sunburst_wordfreq(wafc_era_soynlp, title='Ageism_Era_Conservative', plot_studio=True, save_name='trend_erac_compare_sunburst.html')
plot_sunburst_wordfreq(wafp_era_soynlp, title='Ageism_Era_Progressive', plot_studio=True, save_name='trend_erap_compare_sunburst.html')
plot_bar_wordfreq(wfc_era_soynlp, figsize=(20,8), num_showkeyword=20, title='Ageism_Era_Conservative', save_local=True, save_name='trend_erac_compare_bar_byword.png')
plot_bar_wordfreq(wfp_era_soynlp, figsize=(20,8), num_showkeyword=20, title='Ageism_Era_Progressive', save_local=True, save_name='trend_erap_compare_bar_byword.png')
plot_bar_wordfreq(wafc_era_soynlp, figsize=(20,8), num_showkeyword=20, title='Ageism_Era_Conservative', save_local=True, save_name='trend_erac_compare_bar_bywordadj.png')
plot_bar_wordfreq(wafp_era_soynlp, figsize=(20,8), num_showkeyword=20, title='Ageism_Era_Progressive', save_local=True, save_name='trend_erap_compare_bar_bywordadj.png')

## 과거 10년 동안의 온라인 검색어 긍부정 데이터량 트랜드

## 고령사회 전후의 온라인 검색어 긍부정 데이터량 트랜드

## 고령사회 전후의 보수 및 진보 긍부정 데이터량 트랜드

## Testing

In [ ]:
pip install transformers

In [80]:
# !git clone https://github.com/e9t/nsmc.git
train = pd.read_table("nsmc/"+"ratings_train.txt")
train = train.dropna().sample(5000).reset_index().iloc[:,1:]
test = pd.read_table("nsmc/"+"ratings_test.txt")
test = test.dropna().sample(5000).reset_index().iloc[:,1:]

def preprocessing_sentence_to_BERTinput(df, tokenizer, colname_data, colname_target=None, seq_len=128,
                                        return_type='tensor'):
    tokens, masks, segments, targets = [], [], [], []
    for i in tqdm(range(len(df))):
        # 변환
        token = tokenizer.encode_plus(df[colname_data][i], max_length=seq_len,
                                      pad_to_max_length=True, truncation=True,
                                      return_attention_mask=True,
                                      add_special_tokens=True)

        # 정리
        tokens.append(token['input_ids'])
        masks.append(token['attention_mask'])
        segments.append(token['token_type_ids'])
        if colname_target != None:
            targets.append(df[colname_target][i])
    
    # array 변환
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    if colname_target != None:
        targets = np.array(targets)
        
    # tensor 변환
    if return_type == 'tensor':
        tokens = tf.convert_to_tensor(tokens, dtype=tf.int32)
        masks = tf.convert_to_tensor(masks, dtype=tf.int32)
        segments = tf.convert_to_tensor(segments, dtype=tf.int32)

    return [tokens, masks, segments], targets

import tensorflow_addons as tfa
from transformers import pipeline, AutoTokenizer, BertTokenizer, BertTokenizerFast
from transformers import AutoModel, AutoModelForTokenClassification, TFBertModel, TFBertForSequenceClassification

MODEL_NAME = 'monologg/kobert'    # 'bert-base-multilingual-cased'
# OPTIMIZER = tfa.optimizers.RectifiedAdam(lr=1.0e-5, weight_decay=0.0025, warmup_proportion=0.05)
OPTIMIZER = tf.keras.optimizers.Adam(lr=1.0e-5)
NUM_LABELS = 2
SEQ_LEN = 64

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
X_train, Y_train = preprocessing_sentence_to_BERTinput(train, tokenizer=tokenizer,
                                                       colname_target='label', colname_data='document', seq_len=SEQ_LEN)
X_test, Y_test = preprocessing_sentence_to_BERTinput(test, tokenizer=tokenizer,
                                                       colname_target='label', colname_data='document', seq_len=SEQ_LEN)

def modeling_BERTsentiment(model_name, optimizer, num_labels=2, seq_len=128):
    # 모델 로딩
    model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=metric)

    return model


model = modeling_BERTsentiment(model_name=MODEL_NAME, optimizer=OPTIMIZER, num_labels=NUM_LABELS, seq_len=SEQ_LEN)
# model.fit(X_train, Y_train, epochs=10, shuffle=True, batch_size=100, validation_data=(X_test, Y_test))

100%|████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 6224.16it/s]


In [ ]:
# https://teddylee777.github.io/huggingface/bert-kor-text-classification/

In [40]:
from transformers import pipeline, AutoTokenizer, BertTokenizer, BertTokenizerFast
from transformers import AutoModel, AutoModelForTokenClassification, TFBertModel, TFBertForSequenceClassification

TASK = 'text-classification'
MODEL_NAME = 'monologg/kobert' # 'bert-base-multilingual-cased'
SENTENCES = df_news['제목'].to_list()[:2]
SEQ_LEN = 32

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer(SENTENCES, max_length=SEQ_LEN, 
          padding='max_length', truncation=True, return_tensors='pt')

# model_sa = pipeline(task=TASK, model=MODEL_NAME)
# result = model_sa(df_news['제목'].to_list()[:100])
# for i, j in zip(df_news['제목'].to_list()[:100], result):
#     print(i,j)

{'input_ids': tensor([[   2,    0,    0,    0,    0,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   2,    0,    0,    0,    0,    0,    0, 6975,    0,    3,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])}

In [91]:
MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

X_train = preprocessing_sentence_to_BERTinput(df_news['제목'][:100], tokenizer=tokenizer)

def modeling_BERTsentiment(model_name, optimizer, seq_len=128):
    # 입력 변환
    tokens = tf.keras.layers.Input((seq_len,), dtype=tf.int32, name='input_ids')
    masks = tf.keras.layers.Input((seq_len,), dtype=tf.int32, name='input_masks')
    segments = tf.keras.layers.Input((seq_len,), dtype=tf.int32, name='input_segments')
    
    # 모델 로딩
    model = TFBertModel.from_pretrained(model_name)
    outputs = model([tokens, masks, segments])[1]
    
    # 모델 구성
    layer = tf.keras.layers.Dense(1, activation='sigmoid', 
                                  kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(outputs)
    model_sentiment = tf.keras.Model([tokens, masks, segments], layer)
    model_sentiment.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
    
    return model_sentiment


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 9121.22it/s]


In [ ]:
# 불러오기
save_name = os.path.join(os.getcwd(), 'Data', 'word_freq_soynlp.csv')
word_freq_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', 'wordadj_freq_soynlp.csv')
wordadj_freq_soynlp = pd.read_csv(save_name)
save_name = os.path.join(os.getcwd(), 'Data', 'word_score_tfidf.csv')
word_score_tfidf = pd.read_csv(save_name)

FREQ_USING = word_freq_soynlp.copy()    # word_freq_soynlp, wordadj_freq_soynlp, word_score_tfidf

dict_word_corr, df_word_corrpair = dict(), pd.DataFrame()
centrality_years = pd.DataFrame()
for year in tqdm(sorted(df[COLNAME_CATEGORY].dt.year.unique())):
    print(year)
    
    # 데이터 분리
    df_sub = df[df[COLNAME_CATEGORY].dt.year == year]
    df_subfreq = FREQ_USING[FREQ_USING[FREQ_USING.columns[0]] == year]
    
    # 단어 벡터화 및 상관관계
    _, word_corr, word_corrpair = preprocessing_wordfreq_to_vectorcorr(df_subfreq.iloc[:,-2:], 
                                                                       df_sub[COLNAME_MINING])
    dict_word_corr[year] = word_corr
#     ## 카테고리 추가
#     word_corrpair['year'] = str(year)
#     word_corrpair = word_corrpair[['year']+list(word_corrpair.columns[:-1])]
#     df_word_corrpair = pd.concat([df_word_corrpair, word_corrpair], axis=0, ignore_index=True)
    
#     # 시각화
#     _, centrality = plot_networkx(df_subfreq, word_corrpair.iloc[:,1:], filter_criteria=None, 
#                                   plot=True, node_size='pagerank')
#     ## 카테고리 추가
#     centrality['year'] = str(year)
#     centrality_years = pd.concat([centrality_years, centrality], axis=0, ignore_index=True)
# G, centrality_total = plot_networkx(FREQ_USING, df_word_corrpair.iloc[:,1:], filter_criteria=None, 
#                                     plot=True, node_size='pagerank')
    
# 저장
# save_name = os.path.join(os.getcwd(), 'Data', 'word_freq_soynlp.csv')
# word_freq_soynlp.to_csv(save_name, index=False, encoding='utf-8-sig')
# save_name = os.path.join(os.getcwd(), 'Data', 'wordadj_freq_soynlp.csv')
# wordadj_freq_soynlp.to_csv(save_name, index=False, encoding='utf-8-sig')
# save_name = os.path.join(os.getcwd(), 'Data', 'word_score_tfidf.csv')
# word_score_tfidf.to_csv(save_name, index=False, encoding='utf-8-sig')

In [ ]:
sub_colnames = list(word_corr.columns[word_corr.sum()>=0.5])
word_corr.loc[sub_colnames,:].loc[:,sub_colnames]

In [ ]:
from pyvis.network import Network

mat_square = nx.from_numpy_array(word_corr.loc[sub_colnames,:].loc[:,sub_colnames].values)
vis = Network(notebook=False, cdn_resources='local', height='1000px', width='1000px')
vis.from_nx(mat_square)
vis.show_buttons(filter_=True)
vis.save_graph('nx.html')
# vis.show('test1.html')

In [ ]:
nx.n

In [ ]:
# https://towardsdatascience.com/chord-diagrams-of-protein-interaction-networks-in-python-9589affc8b91
    
import nxviz as nv
from nxviz import annotate, highlights
from nxviz.plots import despine, rescale, respine

def plot_nxviz(df_freq, df_pairweight, filter_criteria=None):
    pass

G, centrality_total = plot_networkx(FREQ_USING, df_word_corrpair, filter_criteria=None, plot=False)
ax = nv.circos(G, group_by="group_node", node_color_by="group_node")
annotate.circos_labels(G, group_by="group_node", radius=np.pi)
annotate.node_colormapping(G, color_by='group_node')
# annotate.node_labels(G, group_by="group_node", sort_by='score')
annotate.circos_group(G, group_by="group_node")

In [ ]:
# 과거 5년 이후 5년 시각화 비교

In [ ]:
nx.from_pandas_edgelist

# Data Preprocessing

## Crawling

In [ ]:
# # 데이터로딩
# df_news = get_data_from_path(FOLDER_LOCATION)
# df = df_news.copy()
# ## 날짜 인식
# df = df[df.Date != 'None'].reset_index().iloc[:,1:]
# df.Date = pd.to_datetime(df.Date)
# ## 중복 처리
# df.drop_duplicates(inplace=True, ignore_index=True)
# ## 전처리
# df.Category = df.Category.apply(lambda x: text_preprocessor(x, del_bracket_content=False))
# df.Title = df.Title.apply(lambda x: text_preprocessor(x, del_bracket_content=True))
# df.Content = df.Content.apply(lambda x: text_preprocessor(x, del_bracket_content=False))
# display(df.head())